In [1]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

In [2]:
dir_path = r"data\SKW01-SF01_CDF_150s_5mGrid_TS0\RAW_DATA"


# Get the parent directory of dir_path
parent_dir = os.path.dirname(os.path.normpath(dir_path))

# Extract the folder name from the parent directory
folder_name = os.path.basename(parent_dir)

# Construct the file paths
node_csv_path = os.path.join(dir_path, f"{folder_name}_nodes.csv")
shell_csv_path = os.path.join(dir_path, f"{folder_name}_shells.csv")
solid_csv_path = os.path.join(dir_path, f"{folder_name}_solids.csv")
nodal_vels_csv_path = os.path.join(dir_path, f"{folder_name}_nodal_velocities.csv")
solid_thickness_csv_path = os.path.join(dir_path, f"{folder_name}_solid_thicknesses.csv")

In [4]:
xyz_round_converter = {col_name: partial(round_to_n_decimals, n=2) for col_name in ['Node_X', 'Node_Y', 'Node_Z']}

def load_csv(file_path, converters=None, index_col=None):
    """
    Load a CSV file into a pandas DataFrame.

    Args:
    file_path (str): Path to the CSV file.
    converters (dict): Dictionary of functions for converting values in certain columns, defaults to None.
    index_col (str): Column to set as index, defaults to None.

    Returns:
    pd.DataFrame: A pandas DataFrame containing the loaded data.
    """
    df = pd.read_csv(file_path, converters=converters)
    if index_col:
        df.set_index(index_col, inplace=True)
    return df

# Define the CSV files and their respective parameters
csv_files = {
    'nodes': {'file_path': node_csv_path, 'converters': xyz_round_converter, 'index_col': 'Node_Label'},
    'shells': {'file_path': shell_csv_path},
    'solids': {'file_path': solid_csv_path},
    'nodal_vels': {'file_path': nodal_vels_csv_path, 'converters': nodal_vels_converter, 'index_col': 'Node_Label'},
    'solids_thick': {'file_path': solid_thickness_csv_path, 'converters': solid_thick_converter}
}

# Use ThreadPoolExecutor to read CSV files in parallel
with ThreadPoolExecutor() as executor:
    # Submit tasks to the executor to read each CSV file in parallel
    future_to_csv = {executor.submit(load_csv, **params): name for name, params in csv_files.items()}
    
    # Collect the DataFrames as they are completed
    for future in concurrent.futures.as_completed(future_to_csv):
        name = future_to_csv[future]
        try:
            data_frame = future.result()
            globals()[f"{name}_df"] = data_frame
        except Exception as exc:
            print(f'{name} generated an exception: {exc}')

NameError: name 'xyz_round_converter' is not defined